In [ ]:
# Objetivo: Treinar um Perceptron (uma camada Dense com 1 neurônio + sigmoid)
# no dataset IMDb para classificar reviews como negativas (0) ou positivas (1).

# dataset escolhido (IMDb - já vem no Keras):
# - Tarefa: classificação binária de sentimento.
# - Tamanho: ~25.000 amostras de treino e ~25.000 de teste.
# - transformação dos textos em números: usei Bag-of-Words multihot com
#   as 20.000 palavras mais frequentes (dimensão = 20.000). Cada review vira
#   um vetor 0/1 indicando se a palavra apareceu.

# O que cada parte do treinamento faz:
# - Camada Dense(1, sigmoid): é o “perceptron”.
# - Loss = binary_crossentropy: mede o erro entre as probabilidades previstas (0 e 1)
#    rótulos 0/1 =perda padrão para classificação binária.
# - Otimizador = adam: ajusta os pesos usando gradiente com taxas adaptativas.
# - Métricas: accuracy (proporção acertos) e F1 (média precisão e recall).
#   No F1, binarizo as probabilidades com threshold 0,5.
# - Treinamento: 50 épocas, batch_size=10, separando 20% do treino para validação.

# Interpretaçãoo dos resultados:
# - Accuracy de treino costuma chegar perto de 1; o modelo aprende bem os padrões lineares.
# - Val_accuracy e val_f1 por volta de 0,88–0,90 são resultados condizentes para BoW + Perceptron.
# - Às vezes a val_loss sobe enquanto val_accuracy/val_f1 ficam estáveis: pode indicar  overfitting leve.
# - Possíveis melhorias :
#   * adicionar regularização L2 na Dense;
#   * usar EarlyStopping monitorando val_f1 e restaurar os melhores pesos;
#   * ajustar o threshold para maximizar o F1 na validação;
#   * trocar multihot por TF-IDF
#Segue o Código:
#-----------------------------------------------------------------------------------------------------------

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import random

# Reprodutibilidade
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Carregar IMDb e mostrar estatísticas básicas do conjunto
num_words = 20000  # tamanho do vocabulário BoW
(x_train_seq, y_train), (x_test_seq, y_test) = keras.datasets.imdb.load_data(num_words=num_words)

print(f"Treino: {len(x_train_seq)} amostras | Teste: {len(x_test_seq)} amostras")
classes, counts = np.unique(y_train, return_counts=True)
print(f"Classes (0=neg,1=pos): {classes} -> {counts}")

lens = list(map(len, x_train_seq))
print(f"Tamanho médio da review (treino): {np.mean(lens):.1f} tokens | mediana: {np.median(lens)}")

# Vetorização: Bag-of-Words multihot (1 se a palavra aparece na review, 0 se não)
def vectorize_sequences(seqs, dimension):
    X = np.zeros((len(seqs), dimension), dtype="float32")
    for i, s in enumerate(seqs):
        X[i, s] = 1.0
    return X

X_train = vectorize_sequences(x_train_seq, num_words)
X_test  = vectorize_sequences(x_test_seq,  num_words)

print(f"Shape X_train: {X_train.shape} | X_test: {X_test.shape}")
print(f"Densidade média (treino): {X_train.mean():.6f}")

# Métrica F1 personalizada
@tf.function
def f1_metric(y_true, y_pred):
    y_true = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    y_pred_bin = tf.cast(y_pred >= 0.5, tf.float32)

    tp = tf.reduce_sum(y_true * y_pred_bin)
    fp = tf.reduce_sum((1.0 - y_true) * y_pred_bin)
    fn = tf.reduce_sum(y_true * (1.0 - y_pred_bin))

    precision = tp / (tp + fp + 1e-7)
    recall    = tp / (tp + fn + 1e-7)
    return 2.0 * precision * recall / (precision + recall + 1e-7)

# Modelo Perceptron (1 neurônio com sigmoid)
model = keras.Sequential([
    keras.Input(shape=(num_words,)),
    layers.Dense(1, activation="sigmoid")
])

# Compilação do modelo
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", f1_metric]
)

model.summary()

# Treinamento
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=10,
    validation_split=0.2,
    verbose=2
)

# Avaliação no teste e métricas adicionais
print("\n[Evaluate] Métricas no conjunto de teste (Keras):")
test_loss, test_acc, test_f1 = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {test_loss:.4f} | Accuracy: {test_acc:.4f} | F1: {test_f1:.4f}")

y_prob = model.predict(X_test, verbose=0).ravel()
y_pred = (y_prob >= 0.5).astype(int)

print("\n[Métricas sklearn no teste]:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f} | F1: {f1_score(y_test, y_pred):.4f}")

print("\nMatriz de confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred, digits=4))


Treino: 25000 amostras | Teste: 25000 amostras
Classes (0=neg,1=pos): [0 1] -> [12500 12500]
Tamanho médio da review (treino): 238.7 tokens | mediana: 178.0
Shape X_train: (25000, 20000) | X_test: (25000, 20000)
Densidade média (treino): 0.006889


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │        20,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,001 (78.13 KB)

 Trainable params: 20,001 (78.13 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
2000/2000 - 7s - 4ms/step - accuracy: 0.8618 - f1_metric: 0.8445 - loss: 0.3930 - val_accuracy: 0.8916 - val_f1_metric: 0.8774 - val_loss: 0.3012
Epoch 2/50
2000/2000 - 3s - 2ms/step - accuracy: 0.9247 - f1_metric: 0.9154 - loss: 0.2346 - val_accuracy: 0.8956 - val_f1_metric: 0.8812 - val_loss: 0.2700
Epoch 3/50
2000/2000 - 3s - 2ms/step - accuracy: 0.9451 - f1_metric: 0.9387 - loss: 0.1825 - val_accuracy: 0.8968 - val_f1_metric: 0.8834 - val_loss: 0.2614
Epoch 4/50
2000/2000 - 4s - 2ms/step - accuracy: 0.9578 - f1_metric: 0.9524 - loss: 0.1495 - val_accuracy: 0.8974 - val_f1_metric: 0.8836 - val_loss: 0.2606
Epoch 5/50
2000/2000 - 3s - 2ms/step - accuracy: 0.9661 - f1_metric: 0.9616 - loss: 0.1255 - val_accuracy: 0.8960 - val_f1_metric: 0.8826 - val_loss: 0.2637
Epoch 6/50
2000/2000 - 4s - 2ms/step - accuracy: 0.9733 - f1_metric: 0.9692 - loss: 0.1070 - val_accuracy: 0.8952 - val_f1_metric: 0.8813 - val_loss: 0.2691
Epoch 7/50
2000/2000 - 3s - 2ms/step - accuracy: 0.9790 - 